In [1]:
# transformers not support NumPy 2.0 yet
!pip install -q numpy~=1.26.4 transformers~=4.46.2
!pip install -q datasets pydantic peft

# 訓練 PII 遮掩模型

In [43]:
import pandas as pd

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  TrainingArguments,
  Trainer,
)
from datasets import load_dataset, DatasetDict
from transformers import DataCollatorForSeq2Seq
from transformers import pipeline

from pydantic import BaseModel
from pprint import pprint

import torch

# 載入 PEFT 相關套件
from peft import LoraConfig, TaskType, get_peft_model

# 檢查是否有 GPU 可以使用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else device)

## 下載資料

In [3]:
# The full `train` split
immutable_dataset = load_dataset("ai4privacy/pii-masking-65k", split="train")

## 資料包含什麼？

In [ ]:
# Reserve 0.05% of the training set for testing
test_dataset = immutable_dataset.train_test_split(
  test_size=0.0005, # 0.05% of the data is used for testing
  shuffle=True, # Ensure that train and validation sets are the same across runs
  )
# Split into 80% training and 20% validation sets
train_dataset = test_dataset['train'].train_test_split(
  test_size=0.2, # 20% of the data is used for validation
  shuffle=True, # Ensure that train and test sets are the same across runs
  )
immutable_dataset = DatasetDict({
  'train': train_dataset['train'],
  'validation': train_dataset['test'],
  'test': test_dataset['test'],
  })
immutable_dataset


In [ ]:
# 保留必要 features: 'masked_text', 'unmasked_text'
dataset = immutable_dataset.remove_columns(['token_entity_labels', 'tokenised_unmasked_text'])
dataset

In [ ]:
# 顯示前 first_n_data 筆資料
first_n_data = 3
pd.set_option('display.max_colwidth', None)
pd.DataFrame(dataset['train'].select(range(first_n_data)))

## 訓練設定

In [7]:
# 訓練相關設定
class Config(BaseModel):
  seed: int = 42
  model_name: str = 'distilbert/distilgpt2' # 使用蒸餾模型，降低參數量
  saved_model_path: str = 'sample_data/saved_encoder_model' # path to save the trained model
  train_batch_size: int = 4 # size of the input batch in training
  eval_batch_size: int = 4 # size of the input batch in evaluation
  epochs: int = 5 # 測試至少 5 個 epochs 才有成效
  lr: float = 2e-5 # learning rate, controls how fast or slow the model learns
  weight_decay: float = 0.01 # weight decay, helps the model stay simple and avoid overfitting by penalizing large weights.

config = Config()


## 先觀察 Fine-tuning 前的表現

### 詠唱格式化 (Prompt Formatting)

先定義我們的詠唱 (Prompt) 格式。為此，我們將創建一個格式化函數。

In [8]:
def instruction_formatter(x):
  text = f'''
    Given the information below, mask the personal identifiable information.

    Input: {x['unmasked_text']}

    Output:
  '''

  return text

In [ ]:
# 透過預訓練模型取得 Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
  config.model_name,
)
# 檢視 Tokenizer，是否存在 PADDING token?
pprint(tokenizer)

In [21]:
# Add PADDING token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

In [22]:
# 載入預訓練模型
generator = pipeline(
  task='text-generation',
  model=config.model_name,
  tokenizer=tokenizer,
  device=device,
)

In [ ]:
# 顯示預訓練模型預測結果
input = instruction_formatter(dataset['test'][0])
response = generator(
  input,
  max_new_tokens=512, # 限制最大生成字數
  repetition_penalty=1.5, # 重複機率, 1~2 之間, 1.0 (no penalty), 2.0 (maximum penalty)
)
print(response[0]['generated_text'])

## 數據預處理

In [34]:
def process_func(x):
  token_ids, attention_mask, labels = [], [], []
  instruction = tokenizer(
    instruction_formatter(x),
    truncation=True)
  # Generate the response and add the end of string token
  response = tokenizer(
    x['masked_text'] + tokenizer.eos_token,
    truncation=True,)
  # Combine the instruction and response
  token_ids = instruction['input_ids'] + response['input_ids']
  attention_mask = instruction['attention_mask'] + response['attention_mask']
  # The labels are the response token ids, but ignore the instruction token ids by setting them to -100
  labels = [-100] * len(instruction['input_ids']) + response['input_ids']

  return {
    'input_ids': token_ids,
    'attention_mask': attention_mask,
    'labels': labels,
  }

In [ ]:
tokenized_dataset = dataset.map(
  process_func,
  batched=False,
  remove_columns=dataset['train'].column_names,
)

In [ ]:
# 檢視輸入資料
print('=== 輸入資料 ===')
print(tokenizer.decode(tokenized_dataset['train']['input_ids'][0]))
print()
print('=== 標註資料 (非 -100) ===')
print(tokenizer.decode(
  list(filter(lambda x: x != -100, tokenized_dataset['train']['labels'][0]))
))


## 訓練模型

您現在可以開始訓練您的模型了！使用 AutoModelForCausalLM 加載預訓練的模型：

In [37]:
model = AutoModelForCausalLM.from_pretrained(
  config.model_name,
)

In [ ]:
# 查看可訓練的參數量
model.print_trainable_parameters()

### LoRA 的訓練策略 - 降維打擊

LoRA（Low-Rank Adaptation）是一種用於訓練大型語言模型的技術，旨在提高訓練效率並減少計算資源的需求。以下是為何需要透過LoRA訓練的一些原因：

降低計算成本：LoRA 通過將模型的權重矩陣分解為低秩矩陣，顯著減少了參數的數量，從而降低了計算成本和內存需求。

加速訓練速度：由於參數數量減少，LoRA 可以加速模型的訓練過程，使得在相同的硬件資源下能夠更快地完成訓練。

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/peft/lora_diagram.png)

In [ ]:
# PEFT 配置
lora_config = LoraConfig(
  task_type=TaskType.CAUSAL_LM,
  # 預設 r (rank) 為 8, target_modules 為 None
)
pprint(lora_config)

In [ ]:
# 取得 PEFT 模型
model = get_peft_model(
  model, # 預訓練模型
  lora_config, # PEFT 配置
)

In [ ]:
# target_modules={'c_attn'}, 限制只有 c_attn 模組受到 PEFT 影響
pprint(lora_config)

In [ ]:
# 取得 PEFT 模型, 限制只有 c_attn 模組受到 PEFT 影響
model

In [ ]:
# 查看可訓練的參數量
model.print_trainable_parameters()

In [ ]:
training_args = TrainingArguments(
  output_dir='sample_data/train_output_pii_masking',
  learning_rate=config.lr,
  per_device_train_batch_size=config.train_batch_size,
  per_device_eval_batch_size=config.eval_batch_size,
  num_train_epochs=config.epochs,
  weight_decay=config.weight_decay,
  eval_strategy='epoch', # 每個 epoch 評估一次
  save_strategy='epoch', # 每個 epoch 儲存一次
  load_best_model_at_end=True,
  report_to='none', # Disable wandb on colab
)

trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_dataset['train'],
  eval_dataset=tokenized_dataset['validation'],
  data_collator=DataCollatorForSeq2Seq(tokenizer, padding=True),
  tokenizer=tokenizer,
)

In [ ]:
# 開始訓練，這可能需要一些時間
trainer.train()

In [ ]:
# 儲存模型
trainer.save_model(config.saved_model_path)

In [ ]:
# 載入新模型
generator = pipeline(
  task='text-generation',
  model=config.saved_model_path,
  device=device,
  tokenizer=tokenizer,
)

In [ ]:
# 顯示新模型預測結果
input = instruction_formatter(dataset['test'][0])
response = generator(
  input,
  max_new_tokens=512, # 限制最大生成字數
  repetition_penalty=1.5, # 重複機率, 1~2 之間, 1.0 (no penalty), 2.0 (maximum penalty)
)
print(response[0]['generated_text'])